In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Set the local path to your fine-tuned model
model_path = r"D:\CODER\Intel_2025\Quiz_Generation\t5_quiz_model_new_finetuned"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the fine-tuned model
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

print("✅ Model and tokenizer loaded successfully!")


c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Model and tokenizer loaded successfully!


In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(r"D:\CODER\Intel_2025\Quiz_Generation\t5_quiz_model_new_finetuned")
tokenizer = AutoTokenizer.from_pretrained(r"D:\CODER\Intel_2025\Quiz_Generation\t5_quiz_model_new_finetuned")

In [16]:
# Define a sample context for question generation
context = "The mitochondria is the powerhouse of the cell."

# Prepare input for the model
input_text = f"Generate a question based on: {context}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate question
output_ids = model.generate(input_ids, max_length=50)
question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Question:", question)


Generated Question: The mitochondria is the powerhouse of the cell.


In [17]:
def generate_mcq_question(context):
    """Generate a structured multiple-choice quiz question with four answer options using the fine-tuned T5 model."""
    input_text = f"Generate a multiple-choice question with four options based on: {context}. Format the output as: 'Question? (A) Option 1 (B) Option 2 (C) Option 3 (D) Option 4'"

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    #input_ids = input_ids.to(model.device)

    # Generate MCQ with controlled output
    output_ids = model.generate(
        input_ids,
        max_length=100,  # Slightly longer for options
        num_beams=5,  # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevents repetitive sequences
        temperature=0.7,  # Adds controlled randomness
        top_k=50,
        top_p=0.95
    )

    question_with_options = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question_with_options

# **Example Input**
#context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
context = "The mitochondria is the powerhouse of the cell."
print(generate_mcq_question(context))


C:\Users\Nakul Limbani\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\Nakul Limbani\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The mitochondria is the powerhouse of the cell.. Format the output as: 'Question?


In [18]:
def test_question_generation():
    context = "The mitochondria is the powerhouse of the cell."
    input_text = f"Generate a question from this passage: {context}"
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print("Generated Question:", question)

test_question_generation()


Generated Question: The mitochondria is the powerhouse of the cell.


In [19]:
def generate_mcq_question(context):
    """Generate a multiple-choice quiz question with four answer options using the fine-tuned T5 model."""
    input_text = (
        f"Generate a multiple-choice question based on: {context}. "
        f"Provide four answer choices labeled as (A), (B), (C), and (D). "
        f"Ensure one is the correct answer."
    )

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(
        input_ids,
        max_length=100,  
        num_beams=5,  
        no_repeat_ngram_size=2,  
        temperature=0.7,  
        top_k=50,
        top_p=0.95
    )

    question_with_options = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question_with_options

context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_mcq_question(context))


The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of what?


In [20]:
def generate_mcq_question(context):
    """Generate a structured multiple-choice quiz question with four answer options using the fine-tuned T5 model."""
    input_text = (
        f"Generate a multiple-choice question based on the following passage: {context}. "
        f"Format the output as follows:\n"
        f"Question: ________?\n"
        f"(A) Option 1\n(B) Option 2\n(C) Option 3\n(D) Option 4\n"
        f"Ensure that one option is correct."
    )

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(
        input_ids,
        max_length=120,  
        num_beams=5,  
        no_repeat_ngram_size=2,  
        temperature=0.7,  
        top_k=50,
        top_p=0.95
    )

    mcq = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return mcq

context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_mcq_question(context))


The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to what?


In [21]:
# Function to filter dataset based on topic/domain
def filter_by_topic(topic, dataset):
    """Filters questions by domain (e.g., Science, History)."""
    return [ex for ex in dataset["train"] if ex["topic"].lower() == topic.lower()]

# Example: Get Science-based questions
science_questions = filter_by_topic("Science", tokenized_datasets)
print(science_questions[:3])  # Display first 3 science-related questions


NameError: name 'tokenized_datasets' is not defined

In [22]:
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')

def generate_distractors(answer, num_distractors=3):
    """Generate similar but incorrect options for a quiz answer."""
    synonyms = set()
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    # Remove correct answer & limit distractors
    distractors = list(synonyms - {answer})[:num_distractors]
    
    # If WordNet doesn't provide enough distractors, add random words (fallback)
    while len(distractors) < num_distractors:
        distractors.append("RandomOption" + str(len(distractors) + 1))

    return distractors

# Example usage
correct_answer = "Mitochondria"
distractors = generate_distractors(correct_answer)
print("Correct Answer:", correct_answer)
print("Options:", [correct_answer] + distractors)


[nltk_data] Downloading package wordnet to C:\Users\Nakul
[nltk_data]     Limbani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Correct Answer: Mitochondria
Options: ['Mitochondria', 'mitochondrion', 'chondriosome', 'RandomOption3']


In [23]:
def generate_mcq(context):
    """Generate a multiple-choice question with four answer options."""
    # Generate the question using the fine-tuned T5 model
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Generate answer choices
    correct_answer = question.split("?")[-1].strip()  # Extracting answer assumption
    distractors = generate_distractors(correct_answer)

    # Shuffle the options
    import random
    options = [correct_answer] + distractors
    random.shuffle(options)

    # Format the MCQ
    formatted_question = f"{question}\n"
    formatted_question += f"(A) {options[0]}\n(B) {options[1]}\n(C) {options[2]}\n(D) {options[3]}"
    
    return formatted_question

# Example Context
context = "The mitochondria is the powerhouse of the cell."
print(generate_mcq(context))


The mitochondria is the powerhouse of the cell.
(A) RandomOption3
(B) The mitochondria is the powerhouse of the cell.
(C) RandomOption2
(D) RandomOption1


In [25]:
def interactive_quiz():
    """Runs an interactive quiz session with dynamically generated questions."""
    context = input("Enter a topic or concept to generate a question: ")
    mcq = generate_mcq(context)
    print("\n" + mcq)

    # Get user answer
    user_answer = input("\nYour answer (A, B, C, or D): ").strip().upper()
    correct_option = mcq.split("\n")[1][4:]  # Extract correct answer

    # Check correctness
    if user_answer in ["A", "B", "C", "D"]:
        if correct_option in mcq.split(f"({user_answer}) ")[-1]:
            print("✅ Correct!")
        else:
            print(f"❌ Incorrect. The correct answer is: {correct_option}")
    else:
        print("Invalid choice. Please enter A, B, C, or D.")

# Run the quiz
interactive_quiz()



Mathematical programming is a part of what type of programming?
(A) RandomOption3
(B) RandomOption1
(C) RandomOption2
(D) 
❌ Incorrect. The correct answer is: RandomOption3


In [29]:
def generate_mcq_question(context):
    """Generate a structured multiple-choice quiz question with four answer options using the fine-tuned T5 model."""
    input_text = (
        f"Generate a multiple-choice question based on the following passage: {context}. "
        f"Format the output as follows:\n"
        f"Question: ________?\n"
        f"(A) Option 1\n(B) Option 2\n(C) Option 3\n(D) Option 4\n"
        f"Ensure that one option is correct."
    )

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(
        input_ids,
        max_length=120,  
        num_beams=5,  
        no_repeat_ngram_size=2,  
        temperature=0.7,  
        top_k=50,
        top_p=0.95
    )

    mcq = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return mcq

context = "The sum of all angles of triangle is 180 degrees."
print(generate_mcq_question(context))


C:\Users\Nakul Limbani\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\Nakul Limbani\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The sum of all angles of triangle is 180 degrees.. Format the output as follows: Question: ________?


In [30]:
def generate_question(context):
    """Generate a question from a given passage using the fine-tuned T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question

context = "The sum of all angles of a triangle is 180 degrees."
question = generate_question(context)
print("Generated Question:", question)


Generated Question: The sum of all angles of a triangle is 180 degrees.


In [31]:
from nltk.corpus import wordnet
import random

def generate_distractors(correct_answer, num_distractors=3):
    """Generate similar but incorrect options for a quiz answer."""
    synonyms = set()
    for syn in wordnet.synsets(correct_answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    # Remove correct answer & limit to distractors
    distractors = list(synonyms - {correct_answer})[:num_distractors]

    # If not enough synonyms, add random numbers as distractors
    while len(distractors) < num_distractors:
        distractors.append(str(random.randint(100, 250)))  # Random incorrect values

    return distractors

# Example usage
correct_answer = "180 degrees"
distractors = generate_distractors(correct_answer)
options = [correct_answer] + distractors
random.shuffle(options)  # Shuffle order

print("Options:")
for idx, opt in enumerate(options, start=1):
    print(f"({chr(64+idx)}) {opt}")


Options:
(A) 215
(B) 163
(C) 180 degrees
(D) 215


In [33]:
def generate_mcq(context):
    """Generate a complete multiple-choice question with correct and incorrect options."""
    question = generate_question(context)
    correct_answer = "180 degrees"  # This can be dynamically extracted based on context
    distractors = generate_distractors(correct_answer)
    
    # Combine correct and incorrect options
    options = [correct_answer] + distractors
    random.shuffle(options)

    # Format final MCQ
    formatted_mcq = f"Question: {question}\n"
    for idx, opt in enumerate(options, start=1):
        formatted_mcq += f"({chr(64+idx)}) {opt}\n"

    return formatted_mcq

print(generate_mcq("The sum of all angles of a triangle is 180 degrees."))
def generate_mcq(context):
    """Generate a complete multiple-choice question with correct and incorrect options."""
    question = generate_question(context)
    correct_answer = "180 degrees"  # This can be dynamically extracted based on context
    distractors = generate_distractors(correct_answer)
    
    # Combine correct and incorrect options
    options = [correct_answer] + distractors
    random.shuffle(options)

    # Format final MCQ
    formatted_mcq = f"Question: {question}\n"
    for idx, opt in enumerate(options, start=1):
        formatted_mcq += f"({chr(64+idx)}) {opt}\n"

    return formatted_mcq

print(generate_mcq("The sum of all angles of a triangle is 180 degrees."))


Question: The sum of all angles of a triangle is 180 degrees.
(A) 235
(B) 132
(C) 114
(D) 180 degrees

Question: The sum of all angles of a triangle is 180 degrees.
(A) 184
(B) 217
(C) 242
(D) 180 degrees



In [34]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load T5 Model
model_name = "valhalla/t5-base-qg-hl"  # Pretrained model for question generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_question(context):
    """Generate a question using a pretrained T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(
        input_ids,
        max_length=50,
        num_beams=5,  # Beam search for stable generation
        temperature=0.7
    )

    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# Example usage
context = "The mitochondria is the powerhouse of the cell."
question = generate_question(context)
print("Generated Question:", question)


C:\Users\Nakul Limbani\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nakul Limbani\.cache\huggingface\hub\models--valhalla--t5-base-qg-hl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.token

: 

In [1]:
import random
import nltk
from nltk.corpus import wordnet
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

# Load necessary models
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
mcq_model_name = "pierreguillou/t5-base-question-generation"
tokenizer = T5Tokenizer.from_pretrained(mcq_model_name)
model = T5ForConditionalGeneration.from_pretrained(mcq_model_name)

nltk.download("wordnet")

def extract_answer(context, question):
    """Extract the correct answer from the passage using a QA model."""
    result = qa_model(question=question, context=context)
    return result["answer"]

def generate_distractors(answer, num_distractors=3):
    """Generate similar but incorrect options for a quiz answer."""
    synonyms = set()
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    distractors = list(synonyms - {answer})[:num_distractors]
    if len(distractors) < num_distractors:
        distractors.extend([str(int(answer.split()[0]) + i) + " degrees" for i in range(1, num_distractors + 1)])

    return distractors[:num_distractors]

def generate_mcq(context):
    """Generate a full MCQ from a given context."""
    # Step 1: Generate the Question
    input_text = f"Generate a multiple-choice question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=100)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Step 2: Extract Correct Answer
    correct_answer = extract_answer(context, question)

    # Step 3: Generate Distractors
    distractors = generate_distractors(correct_answer)

    # Step 4: Shuffle Options
    options = [correct_answer] + distractors
    random.shuffle(options)

    # Step 5: Format MCQ
    formatted_mcq = f"❓ {question}\n(A) {options[0]}\n(B) {options[1]}\n(C) {options[2]}\n(D) {options[3]}"
    
    return formatted_mcq, correct_answer

# **Example Input**
context = "The sum of all angles of a triangle is 180 degrees."
mcq, answer = generate_mcq(context)
print(mcq)
print("\n✅ Correct Answer:", answer)


c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Nakul Limbani\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nakul Limbani\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activa

RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.